In [2]:
#importando as bibliotecas

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import plotly.graph_objs as go

import pandas as pd
import math
from datetime import datetime

In [3]:
#lendo os dados
pop = pd.read_csv("datasets/population.csv")

#removendo as colunas que nao serao uteis ao dashboard
columns_removed = pop.columns[[0,1,2,3,4,14,15,16,17]]
pop_droped = pop.drop(columns_removed, axis=1)

In [4]:
# creating the dataframe urbanization
columns = ['continent', 'year', 'population']
urbanization = pd.DataFrame(columns=columns)

# creating the Series of years
years = pd.Series([2022, 2020, 2015, 2010, 2000, 1990, 1980, 1970])

new_rows = []

# iterate over the rows of pop_droped dataframe
for row in pop_droped.index:

    continent = pop_droped.loc[row, 'Continent']
    
    x=1

    # iterate over the years
    for year in years:
        
        population = pop_droped.iloc[row, x]
        # append the data to the urbanization dataframe
        new_rows.append({'continent': continent, 'year': year, 'population': population})
        x+=1
        
df_new_rows = pd.DataFrame(new_rows)

urbanization = pd.concat([urbanization, df_new_rows], ignore_index=True)


In [5]:
urbanization['year'] = pd.to_datetime(urbanization['year'], format = '%Y')
urbanization['population'] = urbanization['population'].astype(int)

In [6]:
urbanization.head(5)

,continent,year,population
0,Asia,2022-01-01,1425887337
1,Asia,2020-01-01,1424929781
2,Asia,2015-01-01,1393715448
3,Asia,2010-01-01,1348191368
4,Asia,2000-01-01,1264099069


In [15]:
#identificando os continentes
continents = ['Africa', 'Asia', 'Oceania', 'Europe', 'North America', 'South America']

#population_by_year é a somatoria da populacao de cada continente por ano
population_by_year = (urbanization.query('continent.isin(@continents)').groupby(['continent','year']).agg({'population':'sum'}).reset_index())

#criando um grafico de linhas para a o crescimento populacional dos continentes
population_by_year_lines = []
for region in population_by_year['continent'].unique():
    current = population_by_year.query('continent ==@region')
    population_by_year_lines += [go.Scatter(x = current['year'], 
                                           y = current['population'], 
                                           mode = 'lines',
                                           name = region)]

#o mesmo gráfico em áreas    
population_by_year_areas = []
for region in population_by_year['continent'].unique():
    current = population_by_year.query('continent==@region')
    population_by_year_areas += [go.Scatter(x = current['year'],
                                            y = current['population'],
                                            mode = 'lines',
                                            stackgroup = 'one',
                                            name = region)]

#o mesmo grafico em disperssão
population_by_year_scatter = []
for region in population_by_year['continent'].unique():
    current = population_by_year.query('continent==@region')
    population_by_year_scatter += [go.Scatter(x = current['year'],
                                              y = current['population'],
                                              mode = 'markers',
                                              name = region
                                             )
                                    ]

In [18]:
#definindo layout
external_stylesheets = ['<https://codepen.io/chriddyp/pen/bWLwgP.css>']
app = dash.Dash(__name__, external_stylesheets = external_stylesheets)
app.layout = html.Div(children=[


    #plotando o gráfico de linhas
    html.Label('Line chart:'),
    dcc.Graph(
        figure = {'data': population_by_year_lines,
                  'layout': go.Layout(xaxis = {'title':'Year'},
                                      yaxis = {'title':'Population'})
                    },
        id = 'population_by_year_lines'
    ),

    #plotando o gráfico de areas
    html.Label('Stacked area chart:'),
    dcc.Graph(
        figure = {'data': population_by_year_areas,
                  'layout': go.Layout(xaxis = {'title': 'Year'},
                                      yaxis = {'title': 'Population'})
                    },
        id = 'population_by_year_areas'
    ),

    #plotando o gráfico de dispersão
    html.Label('Scatter ploy:'),
    dcc.Graph(
        figure = {'data':population_by_year_scatter,
                  'layout': go.Layout(xaxis = {'title': 'Year'},
                                      yaxis = {'title':'Population'})
                    },
        id = 'population_by_year_scatter'
    ),

])

# lógica do dashboard
if __name__ == '__main__':
    app.run_server(debug=True)